## Introduction to our first task: 'Dogs vs Cats'

In [1]:
import tensorflow as tf

/home/jainrashi2804/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9144428792294901886
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11310504346
locality {
  bus_id: 1
}
incarnation: 17144102974020921219
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

/home/jainrashi2804/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, #preprocess_input: zero-centers our image data using the mean channel values from the training dataset. This is an extremely important step that, if skipped, will cause all the predicted probabilities to be incorrect. This mean centering is what’s called data normalization, a fundamental concept in machine learning.
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
base_model = ResNet50(weights='imagenet', include_top=False) #include_top: whether to include the fully-connected layer at the top of the network.
x = base_model.output                                        #example for include_top:  input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be  (299, 299, 3). It should have exactly 3 inputs channels, and width and height should be no smaller than 71. E.g.  (150, 150, 3) would be one valid value.
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False       #You can pass a trainable argument (boolean) to a layer constructor to set a layer to be non-trainable:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
%%time
#workers: maximum number of processes to spin up when using process based threading
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers = 4,validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 210s 584ms/step - loss: 0.2174 - acc: 0.9464 - val_loss: 0.1006 - val_acc: 0.9768
Epoch 2/3
359/359 [==============================] - 207s 576ms/step - loss: 0.0862 - acc: 0.9711 - val_loss: 0.0878 - val_acc: 0.9768
Epoch 3/3
359/359 [==============================] - 207s 576ms/step - loss: 0.0692 - acc: 0.9758 - val_loss: 0.0683 - val_acc: 0.9830
CPU times: user 14min 56s, sys: 49.4 s, total: 15min 45s
Wall time: 10min 23s


In [10]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [11]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 253s 706ms/step - loss: 0.0798 - acc: 0.9729 - val_loss: 7.7805 - val_acc: 0.5096
CPU times: user 5min 59s, sys: 24.5 s, total: 6min 24s
Wall time: 4min 14s
